In [103]:
from bs4 import BeautifulSoup as soup

import requests
import time
import random

search_text = "golang junior"

In [104]:
def wait ():
    sleep_time = {'min': 0.1, 'max': 1}
    time.sleep(sleep_time["min"]+(random.randrange(0, 10)*0.1*(sleep_time["max"]-sleep_time["min"])))

url = "https://nn.hh.ru/search/vacancy"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:91.0) Firefox/91.0",
    "Accept": "*/*",
    "Accept-Language": "ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3",
    "Connection": "keep-alive"
}

params = {
    "clusters": False,
    "ored_clusters": False,
    "enable_snippets": False,
    "salary": None,
    "st": "searchVacancy",
    "text": search_text,
    "page": 0
}

def get_max_page_count():
    
    response = requests.get(url, headers = headers, params = params)
    response_parsed = soup(response.text, 'html.parser')
    

    pager_buttons = response_parsed.select("div[data-qa='pager-block'] a[class='bloko-button'][data-qa='pager-page']")  
    
    max_page_number = 0
    
    for pager_button in (pager_buttons or []):
        number = int(pager_button.get_text())
        max_page_number = number if number > max_page_number else max_page_number
        
    wait()
    print("max page count: ",max_page_number)
    return max_page_number


vacancys_data = []
vacancys_on_page = False 
vacancys_page_count = get_max_page_count()

while params['page'] < vacancys_page_count:
    
    while not vacancys_on_page:
        
        response = requests.get(url, headers = headers, params = params)
        response_parsed = soup(response.text, 'html.parser')

        # ищем блоки с вакансиями и выдергиваем url
        vacancys_on_page = response_parsed.select("div[class=vacancy-serp-item]")  
        vacancys_on_page = (vacancys_on_page if len(vacancys_on_page or [])>0 else False)

        # если нашли вакансии - составляем под них структуры с данными
        if vacancys_on_page:
            for vacancy_block in vacancys_on_page:
                link_container = vacancy_block.select("a[class='bloko-link']")
                vacancys_data.append({"url": link_container[0]['href']})

        print("page ", params['page'], ", count: ", len(vacancys_on_page or []) )
        
        # симулируем
        wait()
        
    vacancys_on_page = False

    params['page'] += 1


max page count:  2
page  0 , count:  0
page  0 , count:  20
page  1 , count:  20


In [105]:
from translate import Translator
import re
import datetime

translator = Translator(to_lang="en", from_lang = "ru")
    
def parse_date (raw_text):
    date_text = re.search(r'\d+\s+[^\d\s]+\s+\d+', raw_text or "").group()
    date_splited = re.split("\s+", date_text)
    translation = translator.translate(date_splited[-2])
    translated_month_words = re.split("\s+", translation)
    translation = translated_month_words[-1]
    try:
        datetime_object = datetime.datetime.strptime(translation, "%B")
    except ValueError:
        return raw_text
    return datetime.date(int(date_splited[-1]), datetime_object.month, int(date_splited[0]))

In [106]:
import codecs

# разбираем страницу вакансии
counter = 1
for data in vacancys_data:
    
    response = requests.get(data['url'], headers = headers)
    response_parsed = soup(response.text, 'html.parser')    
   
    # вытаскиваем заголовок
    
    title = response_parsed.select("h1[data-qa = 'vacancy-title']")
    data["title"] = str(title[0].get_text()) if len(title)>0 else False
    
    # зарплата бля(
    
    salary_span = response_parsed.select("span[data-qa = 'bloko-header-2'][class = 'bloko-header-2 bloko-header-2_lite']")
    
    #print(counter," of ",len(vacancys_data)," : ", salary_span.get_text())
    counter += 1
    
    data["salary"] = str(salary_span[0].get_text()) if len(salary_span)>0 else False
    
    # название компании
    
    company_name_container = response_parsed.select("a[data-qa = 'vacancy-company-name'][class = 'vacancy-company-name'] > span[data-qa = 'bloko-header-2']")
    
    data["company_name"] = company_name_container[0].get_text() if len(company_name_container)>0 else False
    
    # проверенный работодатель
    
    verified_company_link = response_parsed.select("div[class = 'vacancy-company-badge'] > a[class = 'bloko-link'][href = 'https://feedback.hh.ru/article/details/id/5951']")
    data["company_verified"] = True if len(verified_company_link)>=1 else False
    
    # адрес 
    
    full_adress = response_parsed.select("span[data-qa = 'vacancy-view-raw-address']") 
    city_or_smth = response_parsed.select("p[data-qa = 'vacancy-view-location']")
    
    city_or_smth = city_or_smth[0] if len(city_or_smth)>0 else False
    full_adress = full_adress[0] if len(full_adress)>0 else False
        
    data["adress"] = ""
    
    if city_or_smth or full_adress:
        data["adress"] = (city_or_smth or full_adress).get_text()
        
    # режим на объекте
    employment_mode = response_parsed.select("p[data-qa = 'vacancy-view-employment-mode']") 
    data["employment_mode"] = employment_mode[0].get_text() if len(employment_mode)>0 else False
    
    # опыт работы 
    experience = response_parsed.select("span[data-qa = 'vacancy-experience']") 
    data["experience"] = experience[0].get_text() if len(experience)>0 else False
    
    # parttime
    parttime = response_parsed.select("p[data-qa = 'vacancy-view-parttime-options']") 
    data["parttime"] = parttime[0].get_text() if len(parttime)>0 else False

    # accept-temporary
    accept_temporary = response_parsed.select("p[data-qa = 'vacancy-view-accept-temporary']") 
    data["accept_temporary"] = accept_temporary[0].get_text() if len(accept_temporary)>0 else False

    # full_description
    description = response_parsed.select("div[class = 'vacancy-section']") 
    data["description"] = description[0].get_text() if len(description)>0 else False
  
    # дата
    date = response_parsed.select("p[class = 'vacancy-creation-time']") 
    data["creation-time"] = date[0].get_text() if len(date)>0 else False
    
    # тэги 
    tags = response_parsed.select("div[class = 'bloko-tag-list'] > div") 
    tags = tags[0] if len(tags)>0 else False
    
    tag_list = []
    if tags:
        for tag in tags.children:
            tag_list.append(tag.get_text())
    
    data["tag"] = tag_list
    
    # на скорую руку подчищаем вывод
    for key in data:
        if type(key) == type(""):
            data[key] = codecs.encode(str(data[key]),'utf8')
            data[key] = codecs.decode(data[key],'utf8')
    
    data["creation-time"] = parse_date(data["creation-time"])
  

           
    print(counter,"of",len(vacancys_data))
    
    # симулируем
    
    wait()    



2 of 40
3 of 40
4 of 40
5 of 40
6 of 40
7 of 40
8 of 40
9 of 40
10 of 40
11 of 40
12 of 40
13 of 40
14 of 40
15 of 40
16 of 40
17 of 40
18 of 40
19 of 40
20 of 40
21 of 40
22 of 40
23 of 40
24 of 40
25 of 40
26 of 40
27 of 40
28 of 40
29 of 40
30 of 40
31 of 40
32 of 40
33 of 40
34 of 40
35 of 40
36 of 40
37 of 40
38 of 40
39 of 40
40 of 40
41 of 40


In [107]:
import pandas as pd

df = pd.DataFrame(columns= list(vacancys_data[0].keys()))

ind = 0
for data in vacancys_data:     
    if (len(df.columns)==len(data)):
        df.loc[ind] = data
        ind+=1

df.to_csv(search_text+".csv", encoding="utf8" )
pd.set_option("display.max_rows",None)
df




,url,title,salary,company_name,company_verified,adress,employment_mode,experience,parttime,accept_temporary,description,creation-time,tag
0,https://nn.hh.ru/vacancy/48170363?from=vacancy...,Backend / Developer Junior Golang,от 170 000 руб. до вычета налогов,ООО ВФМ технолоджи,True,"Москва, Тверская улица, 16с3","Полная занятость, гибкий график",1–3 года,False,False,Приглашаем бекенд-разработчикa Golang / GO / J...,Вакансия опубликована 4 октября 2021 в Москве,"['Git', 'Golang', 'ООП', 'Linux', 'Go', 'Atlas..."
1,https://nn.hh.ru/vacancy/48183535?from=vacancy...,Junior Golang developer,от 35 000 до 55 000 руб. на руки,Social Media Holding,True,"Тульская, Москва, Духовской переулок, 17с11","Полная занятость, полный день",1–3 года,False,False,О нас: Компания занимается комплексным продви...,Вакансия опубликована 4 октября 2021 в Москве,"['Go', 'Docker', 'Git', 'Хорошая обучаемость',..."
2,https://nn.hh.ru/vacancy/48170360?from=vacancy...,Backend / Developer Junior Golang,от 110 000 руб. до вычета налогов,ООО ВФМ технолоджи,True,Новосибирск,"Полная занятость, гибкий график",1–3 года,False,False,"Приглашаем бекенд-разработчикa Golang / GO /, ...",Вакансия опубликована 4 октября 2021 в Новосиб...,"['Git', 'Golang', 'ООП', 'Linux', 'Go', 'Backe..."
3,https://nn.hh.ru/vacancy/48460938?from=vacancy...,Младший (Junior) Go (Golang) разработчик в отд...,от 40 000 руб. на руки,amoCRM,True,"Марксистская, Таганская, Таганская, Москва, Ни...","Полная занятость, полный день",не требуется,False,False,Кого мы ищем:Мы ищем начинающих Golang разрабо...,Вакансия опубликована 6 октября 2021 в Москве,"['go', 'golang']"
4,https://nn.hh.ru/vacancy/48514937?from=vacancy...,Junior Golang Developer,з/п не указана,Nexters,True,"Курская, Курская, Курская, Чкаловская, Москва,...","Полная занятость, удаленная работа",1–3 года,False,False,\n\n\n\n\n\n\nNexters - разработчик и издатель...,Вакансия опубликована 5 октября 2021 в Москве,"['SQL', 'Python', 'Java', 'Git', 'C++', 'Golang']"
5,https://nn.hh.ru/vacancy/48563112?from=vacancy...,Junior Golang Developer,до 130 000 руб. на руки,FBS Inc.,True,Санкт-Петербург,"Полная занятость, полный день",1–3 года,False,False,Финтех-компания FBS – разработчик программного...,Вакансия опубликована 6 октября 2021 в Санкт-П...,['Golang']
6,https://nn.hh.ru/vacancy/48297224?from=vacancy...,Junior/Middle Golang Developer,з/п не указана,MicroAvia,True,"Санкт-Петербург, Лахтинский проспект, 85к2","Полная занятость, полный день",1–3 года,False,False,В МикроАвиа мы строим беспилотное будущее уже ...,Вакансия опубликована 6 октября 2021 в Санкт-П...,"['Git', 'Golang', 'Python', 'Linux', 'Go']"
7,https://nn.hh.ru/vacancy/47866523?from=vacancy...,Младший Golang разработчик,от 50 000 до 70 000 руб. на руки,АО Эшелон Технологии,True,"Москва, Электрозаводская улица, 24","Полная занятость, гибкий график",1–3 года,False,False,В нашей расширяющейся команде открыта вакансия...,Вакансия опубликована 4 октября 2021 в Москве,"['Go', 'GoLang', 'NoSQL', 'Базы данных', 'Java..."
8,https://nn.hh.ru/vacancy/45450494?from=vacancy...,Junior/Middle Golang Developer,от 50 000 до 200 000 руб. на руки,Productivity Inside,True,"Новосибирск, Советский район, микрорайон Акаде...","Полная занятость, гибкий график",не требуется,False,False,"Productivity Inside - IT компания, создающая в...",Вакансия опубликована 6 октября 2021 в Новосиб...,"['gRPC', 'REST', 'WebSocket', 'Go', 'MongoDB',..."
9,https://nn.hh.ru/vacancy/47286109?from=vacancy...,Junior Backend Developer,з/п не указана,ООО Смартвэй,True,"Пенза, улица Плеханова, 34","Полная занятость, полный день",1–3 года,False,False,Кто мы: Smartway - первый разработчик IT решен...,Вакансия опубликована 6 октября 2021 в Пензе,"['MS SQL', 'C#', '.NET Framework', 'ASP.NET', ..."


In [8]:
from collections import Counter

vacancy_names = df.tag # change here to change source of data/words etc

all_tags = []
for tag in vacancy_names:
    all_tags.append(' '.join(tag))
    
cloud = Counter(all_tags)
from wordcloud import WordCloud, STOPWORDS
stopwords = set(STOPWORDS)
cloud = ''
for x in list(all_tags):
    cloud+=x+' '
wordcloud = WordCloud(width = 800, height = 800, 

                stopwords = stopwords, 
                min_font_size = 8,background_color='white'
                     ).generate(cloud)

import matplotlib.pylab as plt
plt.figure(figsize = (16, 16)) 
plt.imshow(wordcloud)
plt.savefig('vacancy_cloud.png')

In [89]:
! git add *
! git commit -m "more stable"
! git push 

The file will have its original line endings in your working directory
The file will have its original line endings in your working directory


[main 50fb442] more stable
 2 files changed, 749 insertions(+), 560 deletions(-)


To https://github.com/russianSlave/hh_ru_parser.git
   85ce832..50fb442  main -> main


In [109]:
import re
import datetime

raw_text = "Вакансия опубликована 24 сентября 2021 в Минске"

date_text = re.search(r'\d+\s+[^\d\s]+\s+\d+', raw_text).group()
date_splited = re.split("\s+", date_text)
print(date_splited)
from translate import Translator
translator = Translator(to_lang="en", from_lang = "ru")
translation = translator.translate(date_splited[-2])

translated_month_words = re.split("\s+", translation)
print(translated_month_words)
translation = translated_month_words[-1]


datetime_object = datetime.datetime.strptime(translation, "%B")
final_date = datetime.date(int(date_splited[-1]), datetime_object.month, int(date_splited[0]))

print (final_date) 

text = "зп 500 000 000 - 600 00 в наносекунду"

result = re.search(r'(\d+\s+)+', text)

print(result[1])

['24', 'сентября', '2021']
['MYMEMORY', 'WARNING:', 'YOU', 'USED', 'ALL', 'AVAILABLE', 'FREE', 'TRANSLATIONS', 'FOR', 'TODAY.', 'NEXT', 'AVAILABLE', 'IN', '23', 'HOURS', '02', 'MINUTES', '08', 'SECONDSVISIT', 'HTTPS://MYMEMORY.TRANSLATED.NET/DOC/USAGELIMITS.PHP', 'TO', 'TRANSLATE', 'MORE']


ValueError: time data 'MORE' does not match format '%B'

In [92]:
import translit

ModuleNotFoundError: No module named 'translit'